In [ ]:
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict
policy_setup = "google_robot"
# model_name = "octo-base"
# policy_setup = "widowx_bridge"
model_name = "octo-base"
from simpler_env.policies.octo.octo_model import OctoInference
model = OctoInference(model_type=model_name, policy_setup=policy_setup, init_rng=0)

In [ ]:
#创建环境
from reference.hy_env_add_manipulation import *
np.set_printoptions(suppress=True, precision=3)
args_list = [#google
    "-e", "GraspMultipleSameObjectsInScene-v0",
    "-c", "arm_pd_ee_delta_pose_align_interpolate_by_planner_gripper_pd_joint_target_delta_pos_interpolate_by_planner",
    "-o", "rgbd",
    "robot",
    "google_robot_static",
    "sim_freq", "@500",
    "control_freq", "@5",
    # "scene_name", "bridge_table_1_v1",
    "rgb_overlay_mode", "background",
    "rgb_overlay_path", "/home/khl/khl/Acmu/SENV/component/data/real_inpainting/google_coke_can_real_eval_1.png",
    "rgb_overlay_cameras", "overhead_camera"
]
args_list_w = [#windowx
    "-e", "PushMultipleDifferentObjectsInScene-v0",
    "-c", "arm_pd_ee_target_delta_pose_align2_gripper_pd_joint_pos",
    "-o", "rgbd",
    "robot",
    "widowx",
    "sim_freq", "@500",
    "control_freq", "@5",
    "scene_name", "bridge_table_1_v1",
    "rgb_overlay_mode", "background",
    "rgb_overlay_path", "/home/khl/khl/Acmu/SENV/component/data/real_inpainting/bridge_real_eval_1.png",
    "rgb_overlay_cameras", "3rd_view_camera"
]
args = parse_args(args_list)
env = setup_environment(args)
reset_options = get_env_reset_options(env, args)


In [4]:
import cv2
def resize(img):
    img=cv2.resize(img,(256,256))
    # img=np.expand_dims(img,axis=0)
    return img
def frames_visualization(frames, video_path):
    # 将视频帧保存为视频文件
    height, width, _ = frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video = cv2.VideoWriter(video_path, fourcc, 10, (width, height))  # 帧率为10
    for frame in frames:
        video.write(frame)
    video.release()
    

def run(env, model,run_id,goal_image):
    obs, reset_info= env.reset(options=reset_options)
    # instruction = env.get_language_instruction()
    instruction="Pick the first coke can"
    model.reset(instruction, goal_image)
    image = get_image(obs, 'overhead_camera', "rgb")
    image = resize(image)
    images = [image]
    predicted_terminated, success, truncated = False, False, False
    timestep = 0
    while not (predicted_terminated or truncated):
        # 模型步进
        raw_action, action = model.step(image)
        predicted_terminated = bool(action["terminate_episode"][0] > 0)
        obs, reward, success, truncated, info = env.step(
            np.concatenate([action["world_vector"], action["rot_axangle"], action["gripper"]])
        )
        print(timestep, info)
        # 更新图像观察
        image = get_image_from_maniskill2_obs_dict(env, obs)
        image = resize(image)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)
        timestep += 1
    success_tag = "_success" if success else ""
    video_filename = f"octo{run_id}{success_tag}.mp4"
    #写一个循环，将images的每一帧转为RGB格式
    for i in range(len(images)):
        images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
    # 将images保存为本地视频
    frames_visualization(images, video_filename)
    
goal_image = cv2.imread("/home/khl/khl/Acmu/SENV/component/saved_frames/frame_000119.png")
goal_image = cv2.cvtColor(goal_image, cv2.COLOR_BGR2RGB)
goal_image = cv2.resize(goal_image, (256, 256))
goal_image = np.expand_dims(goal_image, axis=0)
for i in range(1, 30):
    run(env, model, i,goal_image)


0 {'elapsed_steps': 1, 'is_grasped': [False, False, False, False, False, False, False, False], 'consecutive_grasp': [0, 0, 0, 0, 0, 0, 0, 0], 'lifted_object': [False, False, False, False, False, False, False, False], 'lifted_object_significantly': [False, False, False, False, False, False, False, False], 'success': False, 'individual_successes': [False, False, False, False, False, False, False, False], 'episode_stats': OrderedDict([('n_lift_significant_0', 0), ('consec_grasp_0', False), ('grasped_0', False), ('n_lift_significant_1', 0), ('consec_grasp_1', False), ('grasped_1', False), ('n_lift_significant_2', 0), ('consec_grasp_2', False), ('grasped_2', False), ('n_lift_significant_3', 0), ('consec_grasp_3', False), ('grasped_3', False), ('n_lift_significant_4', 0), ('consec_grasp_4', False), ('grasped_4', False), ('n_lift_significant_5', 0), ('consec_grasp_5', False), ('grasped_5', False), ('n_lift_significant_6', 0), ('consec_grasp_6', False), ('grasped_6', False), ('n_lift_significa

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


119 {'elapsed_steps': 120, 'is_grasped': [False, False, False, False, False, False, False, False], 'consecutive_grasp': [0, 0, 0, 0, 0, 0, 0, 0], 'lifted_object': [False, False, False, False, False, False, False, False], 'lifted_object_significantly': [False, False, False, False, False, False, False, False], 'success': False, 'individual_successes': [False, False, False, False, False, False, False, False], 'episode_stats': OrderedDict([('n_lift_significant_0', 0), ('consec_grasp_0', False), ('grasped_0', False), ('n_lift_significant_1', 0), ('consec_grasp_1', False), ('grasped_1', False), ('n_lift_significant_2', 0), ('consec_grasp_2', False), ('grasped_2', False), ('n_lift_significant_3', 0), ('consec_grasp_3', False), ('grasped_3', False), ('n_lift_significant_4', 0), ('consec_grasp_4', False), ('grasped_4', False), ('n_lift_significant_5', 0), ('consec_grasp_5', False), ('grasped_5', False), ('n_lift_significant_6', 0), ('consec_grasp_6', False), ('grasped_6', False), ('n_lift_signi

KeyboardInterrupt: 